# Effective AI Applications: Combine Both RAG & Long Context Window LLMs

## RAG Application Using LlamaIndex & Long Context Model Jamba-Instruct

### Install LlamaIndex & AI21

In [6]:
!pip install llama-index
!pip install -U ai21
!pip install llama-index-llms-ai21

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.2/48.2 kB 7.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 68.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 54.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 12.6 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 28.8 MB/s eta 0:00:0000:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 147.9/147.9 kB 3.8 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 45.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 187.4/187.4 kB 23.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 20.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 18.5 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 373.5/373.5 kB 48.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━

### Import the libraries & dependencies 

In [9]:
import os
from llama_index.core.llama_dataset import download_llama_dataset
from llama_index.core.llama_pack import download_llama_pack
from llama_index.core import VectorStoreIndex
from llama_index.core import SimpleDirectoryReader
from llama_index.llms.ai21 import AI21

In [11]:
os.environ['OPENAI_API_KEY'] = 'Add Your OpneAI API Key' # For embeddings
os.environ['AI21_API_KEY'] = 'Add Your AI21 API Key' # For the generation

### Setup jamba instruct as the LLM

In [12]:
llm = AI21(
    model='jamba-instruct',
    temperature=0,
    max_tokens=2000
)

### Get the data - download 10k forms from AMZN from the last five years

In [13]:
os.mkdir("data")
!wget 'https://d18rn0p25nwr6d.cloudfront.net/CIK-0001018724/c7c14359-36fa-40c3-b3ca-5bf7f3fa0b96.pdf' -O 'data/amazon_2023.pdf'
!wget 'https://d18rn0p25nwr6d.cloudfront.net/CIK-0001018724/d2fde7ee-05f7-419d-9ce8-186de4c96e25.pdf' -O 'data/amazon_2022.pdf'
!wget 'https://d18rn0p25nwr6d.cloudfront.net/CIK-0001018724/f965e5c3-fded-45d3-bbdb-f750f156dcc9.pdf' -O 'data/amazon_2021.pdf'
!wget 'https://d18rn0p25nwr6d.cloudfront.net/CIK-0001018724/336d8745-ea82-40a5-9acc-1a89df23d0f3.pdf' -O 'data/amazon_2020.pdf'
!wget 'https://d18rn0p25nwr6d.cloudfront.net/CIK-0001018724/4d39f579-19d8-4119-b087-ee618abf82d6.pdf' -O 'data/amazon_2019.pdf'

--2024-09-10 16:44:38--  https://d18rn0p25nwr6d.cloudfront.net/CIK-0001018724/c7c14359-36fa-40c3-b3ca-5bf7f3fa0b96.pdf
Resolving d18rn0p25nwr6d.cloudfront.net (d18rn0p25nwr6d.cloudfront.net)... 99.84.178.109, 99.84.178.77, 99.84.178.193, ...
Connecting to d18rn0p25nwr6d.cloudfront.net (d18rn0p25nwr6d.cloudfront.net)|99.84.178.109|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 800598 (782K) [application/pdf]
Saving to: ‘data/amazon_2023.pdf’

data/amazon_2023.pd 100%[===================>] 781.83K  --.-KB/s    in 0.02s   

2024-09-10 16:45:42 (39.5 MB/s) - ‘data/amazon_2023.pdf’ saved [800598/800598]

--2024-09-10 16:45:46--  https://d18rn0p25nwr6d.cloudfront.net/CIK-0001018724/d2fde7ee-05f7-419d-9ce8-186de4c96e25.pdf
Resolving d18rn0p25nwr6d.cloudfront.net (d18rn0p25nwr6d.cloudfront.net)... 99.84.178.109, 99.84.178.193, 99.84.178.124, ...
Connecting to d18rn0p25nwr6d.cloudfront.net (d18rn0p25nwr6d.cloudfront.net)|99.84.178.109|:443... connected.
HTTP request s

### Setup the index 

In [14]:
file_list = [os.path.join("data", f) for f in os.listdir("data")]

amzn_10k_docs = SimpleDirectoryReader(input_files=file_list).load_data()
index = VectorStoreIndex.from_documents(documents=amzn_10k_docs)

# Build a query engine
default_query_engine = index.as_query_engine(llm)

### Let’s enter a query to make sure our RAG system is working.

In [17]:
answer = default_query_engine.query("What was the company's revenue in 2021?")
print(answer.response)

The company's revenue in 2021 was $469,822 million.


### We can see there’s a problem, as our RAG system isn't answering correctly

In [18]:
answer = default_query_engine.query("What was the company's revenue in 2023?")
print(answer.response)

The company's revenue in 2023 was not explicitly mentioned in the provided context. However, it is mentioned that the company's operating income increased to $36.9 billion in 2023, compared to $12.2 billion in 2022.


That’s because the default amount of retrieved chunks is rather small. This makes the whole system prone to errors and failing to capture information that is indeed located in the documents.

### Let’s build a new query engine on top of our existing index and try the query that failed before.
We can increase the number of retrieved chunks from just a few (default value) to 100 and vastly improve the entire RAG system.

In [19]:
# Large amount of chunks in the retrieval process
extended_query_engine = index.as_query_engine(llm,
                                              similarity_top_k=100)

answer = extended_query_engine.query("What was the company's revenue in 2023?")
print(answer.response)

The company's revenue in 2023 was $574.785 million.


In [21]:
answer = default_query_engine.query("Was there a stock split in the last five years?")
print(answer.response)

No, there was no stock split in the last five years.


In [22]:
answer = extended_query_engine.query("Was there a stock split in the last five years?")
print(answer.response)

Yes, there was a stock split in the last five years. On May 27, 2022, Amazon.com, Inc. effected a 20-for-1 stock split of its common stock.


This way, by combining both RAG and long cntext window LLMs, we can build highly effective AI applications.